In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/pickus91/HRV.git

Cloning into 'HRV'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 250 (delta 126), reused 250 (delta 126), pack-reused 0 (from 0)
Receiving objects: 100% (250/250), 3.11 MiB | 29.49 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [3]:
!cat /content/HRV/timeDomain.py
!cat /content/HRV/frequencyDomain.py

# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 13:59:24 2017

@author: picku
"""

import numpy as np

def timeDomain(NN):
    
    L = len(NN)    
    ANN = np.mean(NN)
    SDNN = np.std(NN)
    SDSD = np.std(np.diff(NN))    
    NN50 = len(np.where(np.diff(NN) > 0.05)[0])    
    pNN50 = NN50/L    
    NN20 = len(np.where(np.diff(NN) > 0.02)[0])
    pNN20 = NN20/L
    rMSSD = np.sqrt((1/L) * sum(np.diff(NN) ** 2))        
    MedianNN = np.median(NN)
    
    timeDomainFeats = {'ANN': ANN, 'SDNN': SDNN,
                       'SDSD': SDSD, 'NN50': NN50,
                       'pNN50': pNN50, 'NN20': NN20,
                       'pNN20': pNN20, 'rMSSD': rMSSD,
                       'MedianNN':MedianNN}
                       
    return timeDomainFeats# -*- coding: utf-8 -*-
"""
Created on Fri Mar  3 12:08:40 2017

@author: picku

"""
import numpy as np
from scipy import interpolate, signal
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import matp

In [4]:
import numpy as np
from scipy.stats import chi2

def calculate_rr_noise(rr_intervals, config=None):
    """
    (1) RR 기반 4가지 노이즈 기법 (3σ, 히스토그램, Poincaré, 변동 범위)을 사용해
        'RR 구간의 노이즈 비율(rr_noise_ratio)'을 계산하고, 이를 가중 평균하여 반환.

    반환:
      rr_noise_ratio (float): 0~1 범위 (RR 기반 노이즈 정도)
      detail_dict_rr (dict) : 각 기법별 노이즈 정보
    """
    if config is None:
        config = {}

    sigma_factor        = config.get('sigma_factor', 3.0)         # 3σ 배수
    hist_threshold      = config.get('hist_threshold', 0.15)      # 최빈값 대비 오차 비율
    ellipse_confidence  = config.get('ellipse_confidence', 0.90)  # Poincaré 신뢰구간(기본 90%)
    var_range_threshold = config.get('var_range_threshold', 2.0)  # 변동 범위 임계값
    weights             = config.get('weights', [2.0, 3.0, 1.0, 0.5])
    # ↑ [w_sigma, w_hist, w_ellipse, w_varrange]

    # RR이 너무 적으면 => 노이즈 100% 처리
    if len(rr_intervals) < 2:
        detail_dict_rr = {
            'sigma_noise_ratio': 1.0,
            'hist_noise_ratio': 1.0,
            'ellipse_noise_ratio': 1.0,
            'varrange_noise_ratio': 1.0
        }
        return 1.0, detail_dict_rr

    # RR 통계값
    mean_rr = np.mean(rr_intervals)
    std_rr  = np.std(rr_intervals)

    # ---------------------------
    # (1) 3σ 기반
    # ---------------------------
    upper_bound = mean_rr + sigma_factor * std_rr
    lower_bound = mean_rr - sigma_factor * std_rr
    sigma_outliers = np.sum((rr_intervals < lower_bound) | (rr_intervals > upper_bound))
    sigma_noise_ratio = sigma_outliers / len(rr_intervals)

    # ---------------------------
    # (2) 히스토그램(최빈값) 기반
    # ---------------------------
    bins = np.arange(np.min(rr_intervals), np.max(rr_intervals) + 0.05, 0.05)
    hist, bin_edges = np.histogram(rr_intervals, bins=bins)
    max_bin_index = np.argmax(hist)
    mode_value = (bin_edges[max_bin_index] + bin_edges[max_bin_index+1]) / 2.0
    deviations = np.abs(rr_intervals - mode_value)
    noise_count_hist = np.sum(deviations > hist_threshold * mode_value)
    hist_noise_ratio = noise_count_hist / len(rr_intervals)

    # ---------------------------
    # (3) Poincaré(타원형 분포) 기반
    # ---------------------------
    ellipse_threshold = chi2.ppf(ellipse_confidence, df=2)
    x = rr_intervals[:-1]
    y = rr_intervals[1:]
    points = np.column_stack((x, y))
    mean_point = np.mean(points, axis=0)
    cov_matrix = np.cov(points, rowvar=False)
    try:
        inv_cov = np.linalg.inv(cov_matrix)
        diff_ = points - mean_point
        m_distances = np.sqrt(np.sum(diff_ @ inv_cov * diff_, axis=1))
        noise_count_ellipse = np.sum(m_distances > ellipse_threshold)
        ellipse_noise_ratio = noise_count_ellipse / len(m_distances)
    except np.linalg.LinAlgError:
        ellipse_noise_ratio = 0.0

    # ---------------------------
    # (4) 변동 범위(MxDMn) 기반
    # ---------------------------
    rr_min = np.min(rr_intervals)
    rr_max = np.max(rr_intervals)
    var_range = rr_max - rr_min
    if var_range > var_range_threshold:
        varrange_noise_ratio = min(1.0, (var_range - var_range_threshold) / var_range)
    else:
        varrange_noise_ratio = 0.0

    # ---------------------------
    # (5) 4가지 기법 가중 평균
    # ---------------------------
    weighted_sum = (weights[0]*sigma_noise_ratio +
                    weights[1]*hist_noise_ratio +
                    weights[2]*ellipse_noise_ratio +
                    weights[3]*varrange_noise_ratio)
    weight_total = sum(weights)
    rr_noise_ratio = weighted_sum / weight_total

    detail_dict_rr = {
        'sigma_noise_ratio': sigma_noise_ratio,
        'hist_noise_ratio': hist_noise_ratio,
        'ellipse_noise_ratio': ellipse_noise_ratio,
        'varrange_noise_ratio': varrange_noise_ratio
    }
    return rr_noise_ratio, detail_dict_rr

In [5]:
import numpy as np
from scipy.stats import chi2

def calculate_amp_peak_noise(ppg_signal, peak_count, config=None):
    """
    (2) 진폭(amplitude) 기반 + 피크(peak) 개수 기반 노이즈 판별
        - (5) 진폭 기반:
            ppg_signal 표준편차가 너무 작거나(amp_threshold_low 미만)
            너무 크면(amp_threshold_high 초과) => 노이즈=1
        - (6) 피크 개수 기반:
            peak_count < min_peak_count or > max_peak_count => 노이즈=1

    반환:
      amplitude_noise_flag (int): 0 or 1
      peakcount_noise_flag (int): 0 or 1
      detail_dict_amppeak (dict)
    """
    if config is None:
        config = {}

    amp_threshold_low  = config.get('amplitude_threshold_low', 100)
    amp_threshold_high = config.get('amplitude_threshold_high', 200000)
    min_peak_count     = config.get('min_peak_count', 5)
    max_peak_count     = config.get('max_peak_count', 200)

    amplitude_noise_flag = 0
    peakcount_noise_flag = 0

    # (5) 진폭 기반 => 표준편차가 범위 밖이면 노이즈=1
    if ppg_signal is not None and len(ppg_signal) > 0:
        amp_std = np.std(ppg_signal)
        if amp_std < amp_threshold_low or amp_std > amp_threshold_high:
            amplitude_noise_flag = 1

    # (6) 피크 개수 기반 => 범위 밖이면 노이즈=1
    if peak_count is not None:
        if (peak_count < min_peak_count) or (peak_count > max_peak_count):
            peakcount_noise_flag = 1

    detail_dict_amppeak = {
        'amplitude_noise_flag': amplitude_noise_flag,
        'peakcount_noise_flag': peakcount_noise_flag
    }
    return amplitude_noise_flag, peakcount_noise_flag, detail_dict_amppeak

In [6]:
import numpy as np
from scipy.stats import chi2

def calculate_ppg_var_diff_noise(ppg_signal, config=None):
    """
    (3) PPG 동적(적응형) 임계값 + 미분 기반 노이즈 계산 (윈도우 없이 전체 샘플 단위)

    - 동적 임계값:
      구간 전체 PPG 신호의 전역 통계를 구한 뒤, factor 배만큼을 임계값으로 사용.
      이 임계값을 초과하는 샘플들의 비율을 0~1로 산출.

    - 미분 기반:
      전 구간 미분 신호의 표준편차(또는 평균 등)에 factor를 곱해 임계값 설정.
      초과 샘플 비율을 0~1로 산출.

    반환:
      ppg_noise_ratio (float): 0~1 (분산+미분 종합 노이즈 정도)
      detail_dict_ppg (dict) : 세부 정보
    """
    if config is None:
        config = {}

    # -------------------------------
    # (A) 파라미터 로드
    # -------------------------------
    use_variance_noise   = config.get('use_variance_noise', True)
    dynamic_factor       = config.get('dynamic_factor', 1.5)   # 전역 통계 대비 배수
    use_derivative_noise = config.get('use_derivative_noise', True)
    dynamic_diff_factor  = config.get('dynamic_diff_factor', 1.5)

    # 최종 가중치 (분산 vs 미분)
    var_diff_weights = config.get('var_diff_weights', [1.0, 5.0])
    var_weight, diff_weight = var_diff_weights[0], var_diff_weights[1]

    # -------------------------------
    # (B) 전역 통계 계산
    # -------------------------------
    if ppg_signal is not None and len(ppg_signal) > 0:
        global_mean = np.mean(ppg_signal)
        global_std  = np.std(ppg_signal)
        global_max  = np.max(ppg_signal)
        n_samples   = len(ppg_signal)
    else:
        global_mean = 0.0
        global_std  = 0.0
        global_max  = 0.0
        n_samples   = 0

    # 미분 신호 전역 통계
    diff_signal = None
    if ppg_signal is not None and len(ppg_signal) > 1:
        diff_signal = np.diff(ppg_signal)
        global_diff_std = np.std(diff_signal)
        n_diff_samples  = len(diff_signal)
    else:
        global_diff_std = 0.0
        n_diff_samples  = 0

    # -------------------------------
    # (C) 분산(동적) 기반 (전체 샘플 단위)
    # -------------------------------
    variance_noise_ratio = 0.0
    if use_variance_noise and n_samples > 0:
        # 전역 평균, 표준편차 계산
        lower_bound = global_mean - dynamic_factor * global_std
        upper_bound = global_mean + dynamic_factor * global_std

        # 범위 밖 샘플 개수
        exceed_count = np.sum((ppg_signal < lower_bound) | (ppg_signal > upper_bound))

        # 전체 샘플 대비 비율
        variance_noise_ratio = exceed_count / n_samples

    # -------------------------------
    # (D) 미분(동적) 기반 (전체 샘플 단위)
    # -------------------------------
    derivative_noise_ratio = 0.0
    if use_derivative_noise and diff_signal is not None and n_diff_samples > 0:
        # 전역 미분 신호의 평균과 표준편차
        global_diff_mean = np.mean(diff_signal)
        # 표준편차 * 배수 => 임계값
        dynamic_diff_threshold = global_diff_std * dynamic_diff_factor

        # "평균 ± 임계값" 범위를 벗어나는 샘플 수
        exceed_diff_count = np.sum(
            np.abs(diff_signal - global_diff_mean) > dynamic_diff_threshold
        )

        # 전체 미분 샘플 대비 비율
        derivative_noise_ratio = exceed_diff_count / n_diff_samples

    # -------------------------------
    # (E) 최종 ppg_noise_ratio (가중 평균)
    # -------------------------------
    total_weight = var_weight + diff_weight
    if total_weight > 0:
        ppg_noise_ratio = (
            var_weight * variance_noise_ratio +
            diff_weight * derivative_noise_ratio
        ) / total_weight
    else:
        ppg_noise_ratio = 0.0

    # -------------------------------
    # (F) detail_dict_ppg
    # -------------------------------
    detail_dict_ppg = {
        'variance_noise_ratio': variance_noise_ratio,
        'derivative_noise_ratio': derivative_noise_ratio,
        'ppg_noise_ratio': ppg_noise_ratio,

        # # 전역 통계 로그용
        # 'global_mean': global_mean,
        # 'global_std':  global_std,
        # 'global_max':  global_max,
        # 'global_diff_std': global_diff_std
    }

    return ppg_noise_ratio, detail_dict_ppg

In [7]:
def calculate_noise(rr_intervals, ppg_signal=None, peak_count=None, config=None):
    """
    (4) 최종 노이즈 계산 함수
        1) RR 기반 노이즈(rr_noise_ratio)
        2) 진폭/피크 노이즈(둘 중 하나라도 1이면 -> 즉시 노이즈)
        3) PPG 분산/미분 노이즈(ppg_noise_ratio)

        - RR vs PPG도 가중 평균하여 combined_noise_ratio
        - amplitude_noise_flag or peakcount_noise_flag가 1이면 max()로 최종 1
    """
    if config is None:
        config = {}

    # -----------------------
    # (A) RR 기반 노이즈
    # -----------------------
    rr_noise_ratio, detail_rr = calculate_rr_noise(rr_intervals, config=config)

    # -----------------------
    # (B) 진폭/피크 노이즈 플래그
    # -----------------------
    amplitude_noise_flag, peakcount_noise_flag, detail_amppeak = \
        calculate_amp_peak_noise(ppg_signal, peak_count, config=config)

    # -----------------------
    # (C) PPG 분산/미분 기반
    # -----------------------
    ppg_noise_ratio, detail_ppg = calculate_ppg_var_diff_noise(ppg_signal, config=config)
    variance_noise_ratio   = detail_ppg['variance_noise_ratio']
    derivative_noise_ratio = detail_ppg['derivative_noise_ratio']

    # -----------------------
    # (D) RR vs PPG 가중치
    # -----------------------
    rr_weight  = config.get('rr_weight', 1.0)
    ppg_weight = config.get('ppg_weight', 1.0)

    if (rr_weight + ppg_weight) > 0:
        combined_noise_ratio = (
            rr_weight * rr_noise_ratio +
            ppg_weight * ppg_noise_ratio
        ) / (rr_weight + ppg_weight)
    else:
        combined_noise_ratio = rr_noise_ratio

    # -----------------------
    # (F) 최종 노이즈 결정
    # -----------------------
    #   - amplitude_noise_flag=1 or peakcount_noise_flag=1 이면 => max=1
    final_noise_ratio = max(
        combined_noise_ratio,
        amplitude_noise_flag,
        peakcount_noise_flag
    )

    # -----------------------
    # (G) detail_dict 합치기
    # -----------------------
    detail_dict = {}
    detail_dict.update(detail_rr)       # RR 기반
    detail_dict.update(detail_amppeak)  # 진폭/피크
    detail_dict.update(detail_ppg)      # PPG 분산/미분

    # 주요 값 기록
    detail_dict['rr_noise_ratio']       = rr_noise_ratio
    detail_dict['variance_noise_ratio'] = variance_noise_ratio
    detail_dict['derivative_noise_ratio'] = derivative_noise_ratio
    detail_dict['ppg_noise_ratio']      = ppg_noise_ratio
    detail_dict['combined_noise_ratio'] = combined_noise_ratio
    detail_dict['amplitude_noise_flag'] = amplitude_noise_flag
    detail_dict['peakcount_noise_flag'] = peakcount_noise_flag
    detail_dict['final_noise_ratio']    = final_noise_ratio

    return final_noise_ratio, detail_dict


def calculate_SQI(rr_intervals, ppg_signal=None, peak_count=None, config=None):
    """
    (5) SQI = 1 - final_noise_ratio
    """
    final_noise_ratio, detail_dict = calculate_noise(
        rr_intervals=rr_intervals,
        ppg_signal=ppg_signal,
        peak_count=peak_count,
        config=config
    )
    sqi = 1 - final_noise_ratio
    return sqi, final_noise_ratio, detail_dict

In [8]:
import pandas as pd
import sys
import os

# 파일 경로
data_path = "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/ppg_total_sorted_j70Iq7TuEUhAH1y9QTyiXawI97N2.csv"

# 저장 경로 (원본 덮어쓰기 or 새 파일 저장)
save_as_new_file = True  # True면 새 파일로 저장, False면 원본 덮어쓰기

# 데이터 로딩
try:
    df = pd.read_csv(data_path)
    print(f"✅ 데이터 로딩 완료: {df.shape}")
except Exception as e:
    print(f"❌ 데이터 로딩 오류: {e}")
    sys.exit(1)

# 열 이름 통일
df = df.rename(columns={'timestamp': 'time', 'ppg_raw': 'value'})

# 1. 중복 횟수 세기
time_counts = df['time'].value_counts()
duplicated_times = time_counts[time_counts > 1]
print(f"\n🔍 중복된 time 값 개수: {len(duplicated_times)}")

# 2. 중복 횟수 분포
dup_distribution = duplicated_times.value_counts().sort_index()
print("\n📊 중복 횟수 분포:")
for count, n_times in dup_distribution.items():
    print(f"  {count}번 중복된 time 값 개수: {n_times}")

# 3. 중복 제거 (keep='first' 또는 'last' 선택 가능)
before = len(df)
df_cleaned = df.drop_duplicates(subset='time', keep='first').reset_index(drop=True)
after = len(df_cleaned)
print(f"\n🧹 중복 제거 완료: {before - after}개 제거됨 (총 {after}개 남음)")

# 4. 저장
if save_as_new_file:
    folder = os.path.dirname(data_path)
    filename = os.path.basename(data_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_deduplicated{ext}"
    new_path = os.path.join(folder, new_filename)
else:
    new_path = data_path  # 원본 덮어쓰기

try:
    df_cleaned.to_csv(new_path, index=False)
    print(f"\n💾 중복 제거된 파일 저장 완료: {new_path}")
except Exception as e:
    print(f"❌ 저장 오류: {e}")


✅ 데이터 로딩 완료: (76553840, 5)

🔍 중복된 time 값 개수: 3482097

📊 중복 횟수 분포:
  2번 중복된 time 값 개수: 489731
  3번 중복된 time 값 개수: 134852
  4번 중복된 time 값 개수: 4173
  5번 중복된 time 값 개수: 1
  7번 중복된 time 값 개수: 17522
  9번 중복된 time 값 개수: 539114
  10번 중복된 time 값 개수: 334720
  12번 중복된 time 값 개수: 514005
  18번 중복된 time 값 개수: 87001
  19번 중복된 time 값 개수: 39971
  22번 중복된 time 값 개수: 296453
  24번 중복된 time 값 개수: 31500
  26번 중복된 time 값 개수: 130
  28번 중복된 time 값 개수: 1
  30번 중복된 time 값 개수: 259449
  31번 중복된 time 값 개수: 89716
  32번 중복된 time 값 개수: 36623
  33번 중복된 time 값 개수: 64570
  34번 중복된 time 값 개수: 10200
  38번 중복된 time 값 개수: 280
  39번 중복된 time 값 개수: 2100
  41번 중복된 time 값 개수: 157871
  42번 중복된 time 값 개수: 48329
  44번 중복된 time 값 개수: 1
  46번 중복된 time 값 개수: 1544
  51번 중복된 time 값 개수: 30172
  59번 중복된 time 값 개수: 6787
  61번 중복된 time 값 개수: 1
  63번 중복된 time 값 개수: 300
  64번 중복된 time 값 개수: 2426
  73번 중복된 time 값 개수: 95453
  74번 중복된 time 값 개수: 13347
  77번 중복된 time 값 개수: 3990
  80번 중복된 time 값 개수: 2027
  102번 중복된 time 값 개수: 66910
  104번 중복된 time

In [9]:
# 전처리 이후
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import scipy.signal as signal
import sys
import warnings

sys.path.append('/content/HRV')  # HRV 모듈 경로 추가

# 불필요한 경고 제거
warnings.simplefilter("ignore", UserWarning)

# Matplotlib 설정 (폰트 문제 방지)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.unicode_minus'] = False

# HRV 모듈 import
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain
# from SQI import calculate_SQI  # SQI 함수도 필요 시 import

# --- 1. PPG 데이터 로딩 ---
# PpgGreen_total.csv 경로
data_path = "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승1/total/PpgGreen_total_강현승1[2024-11-28~2024-12-11].csv"
# data_path = "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/ppg_total_sorted_j70Iq7TuEUhAH1y9QTyiXawI97N2_deduplicated.csv"

try:
    df = pd.read_csv(data_path)
    print(f"✅ 데이터 로딩 완료: {df.shape}")
except Exception as e:
    print(f"❌ 데이터 로딩 오류: {e}")
    sys.exit(1)

# --- 2. 열 이름 통일 ---
# timestamp → time, ppg_raw → value
df = df.rename(columns={'timestamp': 'time', 'ppg_raw': 'value'})

# --- 3. 중복 시간값 제거 ---
before_drop = len(df)
df = df.drop_duplicates(subset='time').reset_index(drop=True)
after_drop = len(df)
print(f"🧹 중복 제거 완료: {before_drop - after_drop}개 제거됨 (총 {after_drop}개 남음)")

# --- 4. 시간 변환 및 정렬 ---
try:
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df = df.sort_values(by='time').reset_index(drop=True)
except Exception as e:
    print(f"❌ 시간 변환 오류: {e}")
    sys.exit(1)

# --- 5. 유효한 60초 구간 찾기 ---
valid_start_times = df[df['time'] <= df['time'].max() - pd.Timedelta(seconds=60)]['time'].tolist()
print(f"✅ 사용 가능한 60초 구간 개수: {len(valid_start_times)}")

if len(valid_start_times) == 0:
    print("⚠️ 데이터가 충분하지 않습니다. 다른 데이터셋을 사용하세요.")
    sys.exit(1)

# --- 6. 랜덤 구간 선택 ---
random_sections = 50
selected_start_times = random.sample(valid_start_times, min(len(valid_start_times), random_sections))

# --- 7. 각 구간에 대해 분석 ---
def bandpass_filter(data, fs, lowcut=0.5, highcut=4.0, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data)

for i, start_time in enumerate(selected_start_times):
    end_time = start_time + pd.Timedelta(seconds=60)
    filtered_df = df[(df['time'] >= start_time) & (df['time'] < end_time)].copy()
    data_count = len(filtered_df)
    print(f"\n🟢 [구간 {i+1}] {start_time} ~ {end_time}, 데이터 개수: {data_count}")

    if data_count < 500:
        print("⚠️ 데이터가 부족한 구간입니다. 스킵합니다.")
        continue

    # 필터링
    fs = 25  # Hz
    filtered_df['filtered'] = bandpass_filter(filtered_df['value'], fs)

    # 피크 검출
    min_distance = int(0.33 * fs)
    peaks, _ = signal.find_peaks(filtered_df['filtered'], distance=min_distance)
    if len(peaks) < 5:
        print("⚠️ 피크가 너무 적습니다. 스킵합니다.")
        continue
    print(f"✅ 검출된 피크 개수: {len(peaks)}")

    # RR interval 계산
    peak_times = filtered_df['time'].iloc[peaks].values
    rr_intervals = np.diff(peak_times) / np.timedelta64(1, 's')
    if len(rr_intervals) < 5:
        print("⚠️ RR interval이 너무 적습니다. 스킵합니다.")
        continue
    print(f"✅ RR intervals 개수: {len(rr_intervals)}")

    # SQI 계산
    try:
        sqi, noise_ratio, detail_dict = calculate_SQI(rr_intervals=rr_intervals,
                                                      ppg_signal=filtered_df['filtered'],
                                                      peak_count=len(peaks))
        print(f"✅ 노이즈 비율: {noise_ratio:.2f}, SQI: {sqi:.2f}")
        print("\n🔎 Detail Dict:")
        for key, val in detail_dict.items():
            print(f"  {key}: {val:.4f}" if isinstance(val, float) else f"  {key}: {val}")
    except Exception as e:
        print(f"❌ SQI 계산 오류: {e}")
        continue

    # HRV 분석
    try:
        time_feats = timeDomain(rr_intervals)
        freq_feats = frequencyDomain(rr_intervals)
    except Exception as e:
        print(f"❌ HRV 분석 오류: {e}")
        continue

    # 그래프 출력
    plt.figure(figsize=(12, 4))
    plt.plot(filtered_df['time'], filtered_df['value'], label='Raw PPG', color='blue')
    plt.title(f"Raw PPG (Interval {i+1})")
    plt.xlabel("Time")
    plt.ylabel("PPG Value")
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 4))
    plt.plot(filtered_df['time'], filtered_df['filtered'], label='Filtered PPG', color='green')
    plt.scatter(filtered_df['time'].iloc[peaks], filtered_df['filtered'].iloc[peaks], color='red', label='Detected Peaks')
    plt.title(f"Filtered PPG with Peaks (Interval {i+1})")
    plt.xlabel("Time")
    plt.ylabel("PPG Value")
    plt.legend()
    plt.show()

    # 주파수 도메인 그래프
    try:
        freq_feats_plot = frequencyDomain(rr_intervals, plot=1)
    except Exception as e:
        print(f"❌ 주파수 영역 HRV 그래프 오류: {e}")

    # 결과 출력
    print("\n✅ Time Domain HRV Features:")
    for k, v in time_feats.items():
        print(f"{k}: {v}")

    print("\n✅ Frequency Domain HRV Features:")
    for k, v in freq_feats.items():
        print(f"{k}: {v}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import numpy as np
import scipy.signal as signal
import sys
import warnings

sys.path.append('/content/HRV')  # HRV 모듈 경로 추가

# 불필요한 경고 제거
warnings.simplefilter("ignore", UserWarning)

# Matplotlib 설정 (폰트 문제 방지)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.unicode_minus'] = False

# HRV 모듈 import
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain

# --- 1. PPG 데이터 로딩 ---
sensor_data = {}

# PpgGreen 센서만 사용
sensor_configs = {'PpgGreen': ['time', 'value']}

# 데이터 경로 (Colab 경로)
data_path = "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승/total/PpgGreen_total.csv"

try:
    # 파일 읽기 및 데이터 저장
    for sensor, columns in sensor_configs.items():
        sensor_files = [f for f in os.listdir(data_path) if f.startswith(sensor)]
        data_frames = []

        for file in sensor_files:
            file_path = os.path.join(data_path, file)
            try:
                df = pd.read_csv(file_path, names=columns, header=0)
                data_frames.append(df)
            except Exception as e:
                print(f"❌ 파일 읽기 오류: {file} - {e}")

        if data_frames:
            sensor_data[sensor] = pd.concat(data_frames, ignore_index=True)

except Exception as e:
    print(f"❌ 데이터 로딩 오류: {e}")
    sys.exit(1)

# --- 2. PPG 데이터 처리 ---
if 'PpgGreen' in sensor_data:
    df = sensor_data['PpgGreen']

    # ✅ 데이터 크기 확인
    print(f"✅ 로드된 PpgGreen 데이터 크기: {df.shape}")

    try:
        # 시간 데이터 변환 및 정렬
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        df = df.sort_values(by='time').reset_index(drop=True)
    except Exception as e:
        print(f"❌ 시간 변환 오류: {e}")
        sys.exit(1)

    # ✅ 유효한 60초 구간 찾기
    valid_start_times = df[df['time'] <= df['time'].max() - pd.Timedelta(seconds=60)]['time'].tolist()

    # ✅ 사용 가능한 60초 구간 개수 확인
    print(f"✅ 사용 가능한 60초 구간 개수: {len(valid_start_times)}")

    if len(valid_start_times) == 0:
        print("⚠️ 데이터가 충분하지 않습니다. 다른 데이터셋을 사용하세요.")
        sys.exit(1)

    # ✅ N개의 랜덤 구간 선택
    random_sections = 50
    selected_start_times = random.sample(valid_start_times, random_sections)

    # --- 3. N개 구간에 대해 분석 ---
    for i, random_start_time in enumerate(selected_start_times):
        end_time = random_start_time + pd.Timedelta(seconds=60)
        filtered_df = df[(df['time'] >= random_start_time) & (df['time'] < end_time)].copy()

        # ✅ 구간별 데이터 개수 확인
        data_count = len(filtered_df)
        print(f"\n🟢 [구간 {i+1}] {random_start_time} ~ {end_time}, 데이터 개수: {data_count}")

        if data_count < 500:
            print("⚠️ 데이터가 부족한 구간입니다. 스킵합니다.")
            continue

        # --- 4. 필터링 ---
        def bandpass_filter(data, fs, lowcut=0.5, highcut=4.0, order=2):
            nyq = 0.5 * fs
            low = lowcut / nyq
            high = highcut / nyq
            b, a = signal.butter(order, [low, high], btype='band')
            return signal.filtfilt(b, a, data)

        fs = 25  # 샘플링 주파수 (Hz)
        filtered_df['filtered'] = bandpass_filter(filtered_df['value'], fs)

        # --- 5. 피크 검출 ---
        min_distance = int(0.33 * fs)   // 약 180bpm
        peaks, _ = signal.find_peaks(filtered_df['filtered'], distance=min_distance)
        peak_count = len(peaks)

        # ✅ 피크 개수 확인
        print(f"✅ 검출된 피크 개수: {len(peaks)}")

        if len(peaks) < 5:
            print("⚠️ 피크가 너무 적습니다. 스킵합니다.")
            continue

        # --- 6. RR interval 계산 ---
        peak_times = filtered_df['time'].iloc[peaks].values
        rr_intervals = np.diff(peak_times) / np.timedelta64(1, 's')

        # ✅ RR interval 개수 확인
        print(f"✅ RR intervals 개수: {len(rr_intervals)}")

        if len(rr_intervals) < 5:
            print("⚠️ RR interval이 너무 적습니다. 스킵합니다.")
            continue

        # --- 7. 노이즈 및 SQI 계산 ---
        sqi, noise_ratio, detail_dict = calculate_SQI(rr_intervals=rr_intervals,
                                                 ppg_signal=filtered_df['filtered'],
                                                 peak_count=peak_count)
        print(f"✅ 노이즈 비율: {noise_ratio:.2f}, SQI: {sqi:.2f}")

        # detail_dict 깔끔하게 출력
        print("\n🔎 Detail Dict:")
        for key, val in detail_dict.items():
            # 값이 float인 경우 소수점 2자리, int인 경우 그냥 출력
            if isinstance(val, float):
                print(f"  {key}: {val:.4f}")
            else:
                print(f"  {key}: {val}")

        # --- 8. HRV 분석 ---
        try:
            timeDomainFeats = timeDomain(rr_intervals)
            freqDomainFeats = frequencyDomain(rr_intervals)
        except Exception as e:
            print(f"❌ HRV 분석 오류: {e}")
            continue

        # --- 9. 그래프 출력 ---
        plt.figure(figsize=(12, 4))
        plt.plot(filtered_df['time'], filtered_df['value'], label='Raw PpgGreen', color='blue')
        plt.title(f"Raw PpgGreen Signal (Interval {i+1}: {random_start_time})")
        plt.xlabel("Time")
        plt.ylabel("PPG Value")
        plt.legend()
        plt.show()

        plt.figure(figsize=(12, 4))
        plt.plot(filtered_df['time'], filtered_df['filtered'], label='Filtered PpgGreen', color='green')
        plt.scatter(filtered_df['time'].iloc[peaks], filtered_df['filtered'].iloc[peaks], color='red', label='Detected Peaks')
        plt.title(f"Filtered PpgGreen with Detected Peaks (Interval {i+1}: {random_start_time})")
        plt.xlabel("Time")
        plt.ylabel("PPG Value")
        plt.legend()
        plt.show()

        # --- 10. HRV 분석 (시간 & 주파수 도메인) ---
        try:
            timeDomainFeats = timeDomain(rr_intervals)
            freqDomainFeats = frequencyDomain(rr_intervals, plot=1)  # plot=1 추가해서 주파수 그래프 자동 출력
        except Exception as e:
            print(f"❌ HRV 분석 오류: {e}")
            continue

        # --- 11. HRV 결과 출력 ---
        print("\n✅ Time Domain HRV Features:")
        for key, value in timeDomainFeats.items():
            print(f"{key}: {value}")

        print("\n✅ Frequency Domain HRV Features:")
        for key, value in freqDomainFeats.items():
            print(f"{key}: {value}")

else:
    print("❌ PpgGreen 데이터가 없습니다.")
    sys.exit(1)

In [ ]:
import os
import pandas as pd
import numpy as np
import sys

# 분석할 폴더 경로들
folder_paths = [
    "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승",
    "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/김형욱",
    "/content/drive/MyDrive/가천대/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/송현"
]

def load_ppg_data_from_folder(folder_path, sensor_prefix="PpgGreen_"):
    """
    폴더 안의 PpgGreen_*.csv 파일을 전부 읽어 하나의 DataFrame으로 합쳐 반환.
    (os.listdir + startswith 사용)
    """
    try:
        all_files = os.listdir(folder_path)
    except FileNotFoundError:
        print(f"❌ 폴더를 찾을 수 없습니다: {folder_path}")
        return None
    except Exception as e:
        print(f"❌ os.listdir 오류: {folder_path}, {e}")
        return None

    # PpgGreen_로 시작하는 파일만 골라서 읽기
    sensor_files = [f for f in all_files if f.startswith(sensor_prefix)]
    if not sensor_files:
        print(f"⚠️ {folder_path} 폴더에 {sensor_prefix} 관련 CSV 파일이 없습니다.")
        return None

    df_list = []
    for file_name in sensor_files:
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_csv(file_path, header=0, names=["time", "value"])
            df_list.append(df)
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {file_name} - {e}")

    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        return combined_df
    else:
        return None

def analyze_1min_intervals(df):
    """
    DataFrame(df)의 time 컬럼(이미 datetime 변환됨)을 이용해
    - 처음부터 끝까지 1분 단위로 쪼갰을 때 몇 개 구간이 나오는지 (interval_count)
    - 각 구간별 데이터 개수의 평균 (avg_data_count)
    를 반환한다.
    """
    df = df.sort_values(by='time').reset_index(drop=True)
    start_time = df['time'].iloc[0]
    end_time = df['time'].iloc[-1]

    one_minute = pd.Timedelta(seconds=60)
    intervals_data_counts = []  # 각 구간의 데이터 개수를 저장

    current = start_time
    while current + one_minute <= end_time:
        # 현재 구간의 끝 시간
        current_end = current + one_minute

        # 해당 구간에 속하는 데이터만 추출
        mask = (df['time'] >= current) & (df['time'] < current_end)
        data_count = mask.sum()  # 구간 내 데이터 개수
        intervals_data_counts.append(data_count)

        current = current_end

    interval_count = len(intervals_data_counts)
    if interval_count > 0:
        avg_data_count = sum(intervals_data_counts) / interval_count
    else:
        avg_data_count = 0

    return interval_count, avg_data_count

# 메인 실행부
if __name__ == "__main__":
    for idx, folder_path in enumerate(folder_paths, start=1):
        folder_name = os.path.basename(folder_path)
        ppg_df = load_ppg_data_from_folder(folder_path, sensor_prefix="PpgGreen_")

        if ppg_df is None or len(ppg_df) == 0:
            print(f"[Folder {idx}] {folder_name} - PpgGreen 데이터 없음 또는 로드 실패\n")
            continue

        # 전체 데이터 크기
        total_data_size = len(ppg_df)

        # time 컬럼을 datetime으로 변환
        try:
            ppg_df['time'] = pd.to_datetime(ppg_df['time'], unit='ms')
        except Exception as e:
            print(f"[Folder {idx}] {folder_name} - time 변환 오류: {e}\n")
            continue

        # 1분 단위 구간 수 및 평균 데이터 개수 계산
        interval_count, avg_data_count = analyze_1min_intervals(ppg_df)

        print(f"[Folder {idx}] {folder_name}")
        print(f" - 전체 데이터 크기: {total_data_size}")
        print(f" - 1분 구간 개수: {interval_count}")
        print(f" - 각 구간 평균 데이터 개수: {avg_data_count:.2f}\n")

    print("모든 폴더 처리 완료!")


[Folder 1] 강현승
 - 전체 데이터 크기: 9480886
 - 1분 구간 개수: 18739
 - 각 구간 평균 데이터 개수: 505.87

[Folder 2] 김형욱
 - 전체 데이터 크기: 17693331
 - 1분 구간 개수: 19646
 - 각 구간 평균 데이터 개수: 900.56

[Folder 3] 송현
 - 전체 데이터 크기: 11261135
 - 1분 구간 개수: 76283
 - 각 구간 평균 데이터 개수: 147.61

모든 폴더 처리 완료!


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy.signal as signal
import sys
import warnings

sys.path.append('/content/HRV')  # HRV 모듈 경로 추가

# 불필요한 경고 제거
warnings.simplefilter("ignore", UserWarning)

# Matplotlib 설정 (폰트 문제 방지)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.unicode_minus'] = False

# HRV 모듈 import
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain

# 사용자 이름
user_name = "강현승"

# ===================================================================
# ----------------- 1. PPG 데이터 로딩 부분 -------------------------
# ===================================================================
data_path = "/content/drive/MyDrive/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승"
sensor_data = {}
sensor_configs = {'PpgGreen': ['time', 'value']}

try:
    for sensor, columns in sensor_configs.items():
        sensor_files = [f for f in os.listdir(data_path) if f.startswith(sensor)]
        data_frames = []
        for file in sensor_files:
            file_path = os.path.join(data_path, file)
            try:
                df_tmp = pd.read_csv(file_path, names=columns, header=0)
                data_frames.append(df_tmp)
            except Exception as e:
                print(f"❌ 파일 읽기 오류: {file} - {e}")
        if data_frames:
            sensor_data[sensor] = pd.concat(data_frames, ignore_index=True)

except Exception as e:
    print(f"❌ 데이터 로딩 오류: {e}")
    sys.exit(1)

if 'PpgGreen' not in sensor_data:
    print("❌ PpgGreen 데이터가 없습니다.")
    sys.exit(1)

df = sensor_data['PpgGreen']
print(f"✅ 로드된 PpgGreen 데이터 크기: {df.shape}")

# 시간 변환 및 정렬
try:
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df = df.sort_values(by='time').reset_index(drop=True)
except Exception as e:
    print(f"❌ 시간 변환 오류: {e}")
    sys.exit(1)

# ===================================================================
# ----------------- 2. 전체 1분 구간 반복 설정 ----------------------
# ===================================================================
min_time = df['time'].min()
max_time = df['time'].max()

chunk_duration = pd.Timedelta(seconds=60)

# 결과 저장 폴더 (이미지 등)
output_dir = "./preprocessing"
os.makedirs(output_dir, exist_ok=True)
img_dir = os.path.join(output_dir, "images")
os.makedirs(img_dir, exist_ok=True)

results = []  # CSV로 내보낼 결과를 저장할 리스트 (각 row를 딕셔너리로)

current_start = min_time
while True:
    current_end = current_start + chunk_duration
    if current_end > max_time:
        break  # 더 이상 1분 구간을 만들 수 없으면 종료

    filtered_df = df[(df['time'] >= current_start) & (df['time'] < current_end)].copy()
    data_count = len(filtered_df)

    if data_count < 500:
        results.append({
            "start_time": current_start,
            "end_time": current_end,
            "data_count": data_count,
            "peak_count": None,
            "rr_count": None,
            "sqi": None,
            "noise_ratio": None,
            # Time/Freq Domain
            "ANN": None, "SDNN": None, "SDSD": None,
            "NN50": None, "pNN50": None, "NN20": None,
            "pNN20": None, "rMSSD": None, "MedianNN": None,
            "VLF_Power": None, "LF_Power": None,
            "HF_Power": None, "LF_HF": None,
            # detail_dict
            "sigma_noise_ratio": None,
            "hist_noise_ratio": None,
            "ellipse_noise_ratio": None,
            "varrange_noise_ratio": None,
            "amplitude_noise_ratio": None,
            "peakcount_noise_ratio": None,
            # 그래프 경로
            "raw_plot_path": None,
            "filtered_plot_path": None,
            "freq_plot_path": None
        })
        current_start = current_end
        continue

    # ===================================================================
    # ----------------- 3. 필터링 및 피크 검출 --------------------------
    # ===================================================================
    def bandpass_filter(data, fs, lowcut=0.5, highcut=5.0, order=2):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return signal.filtfilt(b, a, data)

    fs = 25
    filtered_df['filtered'] = bandpass_filter(filtered_df['value'], fs)

    min_distance = int(0.5 * fs)
    peaks, _ = signal.find_peaks(filtered_df['filtered'], distance=min_distance)
    peak_count = len(peaks)

    if peak_count < 5:
        results.append({
            "start_time": current_start,
            "end_time": current_end,
            "data_count": data_count,
            "peak_count": peak_count,
            "rr_count": None,
            "sqi": None,
            "noise_ratio": None,
            # Time/Freq Domain
            "ANN": None, "SDNN": None, "SDSD": None,
            "NN50": None, "pNN50": None, "NN20": None,
            "pNN20": None, "rMSSD": None, "MedianNN": None,
            "VLF_Power": None, "LF_Power": None,
            "HF_Power": None, "LF_HF": None,
            # detail_dict
            "sigma_noise_ratio": None,
            "hist_noise_ratio": None,
            "ellipse_noise_ratio": None,
            "varrange_noise_ratio": None,
            "amplitude_noise_ratio": None,
            "peakcount_noise_ratio": None,
            # 그래프 경로
            "raw_plot_path": None,
            "filtered_plot_path": None,
            "freq_plot_path": None
        })
        current_start = current_end
        continue

    # ===================================================================
    # ----------------- 4. RR intervals & 노이즈(SQI) 계산 --------------
    # ===================================================================
    peak_times = filtered_df['time'].iloc[peaks].values
    rr_intervals = np.diff(peak_times) / np.timedelta64(1, 's')
    rr_count = len(rr_intervals)

    if rr_count < 5:
        results.append({
            "start_time": current_start,
            "end_time": current_end,
            "data_count": data_count,
            "peak_count": peak_count,
            "rr_count": rr_count,
            "sqi": None,
            "noise_ratio": None,
            # Time/Freq Domain
            "ANN": None, "SDNN": None, "SDSD": None,
            "NN50": None, "pNN50": None, "NN20": None,
            "pNN20": None, "rMSSD": None, "MedianNN": None,
            "VLF_Power": None, "LF_Power": None,
            "HF_Power": None, "LF_HF": None,
            # detail_dict
            "sigma_noise_ratio": None,
            "hist_noise_ratio": None,
            "ellipse_noise_ratio": None,
            "varrange_noise_ratio": None,
            "amplitude_noise_ratio": None,
            "peakcount_noise_ratio": None,
            # 그래프 경로
            "raw_plot_path": None,
            "filtered_plot_path": None,
            "freq_plot_path": None
        })
        current_start = current_end
        continue

    # >>> SQI 계산 (detail_dict 포함해서 3개 언패킹)
    sqi, noise_ratio, detail_dict = calculate_SQI(rr_intervals=rr_intervals,
                                                 ppg_signal=filtered_df['filtered'],
                                                 peak_count=peak_count)

    # ===================================================================
    # ----------------- 5. HRV 분석 (시간 & 주파수) ---------------------
    # ===================================================================
    try:
        timeDomainFeats = timeDomain(rr_intervals)
        freqDomainFeats = frequencyDomain(rr_intervals, plot=0)
    except Exception as e:
        print(f"❌ HRV 분석 오류: {e}")
        timeDomainFeats, freqDomainFeats = {}, {}

    # ===================================================================
    # ----------------- 6. 그래프 저장 (raw, filtered, freq) ------------
    # ===================================================================
    raw_fig, raw_ax = plt.subplots(figsize=(12, 4))
    raw_ax.plot(filtered_df['time'], filtered_df['value'], label='Raw PpgGreen', color='blue')
    raw_ax.set_title(f"Raw PpgGreen (Interval: {current_start} ~ {current_end})")
    raw_ax.set_xlabel("Time")
    raw_ax.set_ylabel("PPG Value")
    raw_ax.legend()

    raw_plot_name = f"chunk_{current_start.strftime('%Y%m%d_%H%M%S')}_raw.png"
    raw_plot_path = os.path.join(img_dir, raw_plot_name)
    plt.savefig(raw_plot_path)
    plt.close(raw_fig)

    filt_fig, filt_ax = plt.subplots(figsize=(12, 4))
    filt_ax.plot(filtered_df['time'], filtered_df['filtered'], label='Filtered PpgGreen', color='green')
    filt_ax.scatter(filtered_df['time'].iloc[peaks], filtered_df['filtered'].iloc[peaks],
                    color='red', label='Detected Peaks')
    filt_ax.set_title(f"Filtered + Peaks (Interval: {current_start} ~ {current_end})")
    filt_ax.set_xlabel("Time")
    filt_ax.set_ylabel("PPG Value")
    filt_ax.legend()

    filtered_plot_name = f"chunk_{current_start.strftime('%Y%m%d_%H%M%S')}_filtered.png"
    filtered_plot_path = os.path.join(img_dir, filtered_plot_name)
    plt.savefig(filtered_plot_path)
    plt.close(filt_fig)

    freq_fig = plt.figure(figsize=(8, 4))
    f, pxx = signal.welch(rr_intervals, fs=1.0, nperseg=len(rr_intervals))
    plt.semilogy(f, pxx, label='RR Spectrum')
    plt.title(f"Frequency Domain (Interval: {current_start} ~ {current_end})")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("PSD")
    plt.legend()

    freq_plot_name = f"chunk_{current_start.strftime('%Y%m%d_%H%M%S')}_freq.png"
    freq_plot_path = os.path.join(img_dir, freq_plot_name)
    plt.savefig(freq_plot_path)
    plt.close(freq_fig)

    # ===================================================================
    # ----------------- 7. CSV 결과용 리스트에 저장 ----------------------
    # ===================================================================
    ANN_val = timeDomainFeats.get('ANN')
    SDNN_val = timeDomainFeats.get('SDNN')
    SDSD_val = timeDomainFeats.get('SDSD')
    NN50_val = timeDomainFeats.get('NN50')
    pNN50_val = timeDomainFeats.get('pNN50')
    NN20_val = timeDomainFeats.get('NN20')
    pNN20_val = timeDomainFeats.get('pNN20')
    rMSSD_val = timeDomainFeats.get('rMSSD')
    MedianNN_val = timeDomainFeats.get('MedianNN')

    VLF_val = freqDomainFeats.get('VLF_Power')
    LF_val = freqDomainFeats.get('LF_Power')
    HF_val = freqDomainFeats.get('HF_Power')
    LF_HF_val = freqDomainFeats.get('LF/HF')

    row_dict = {
        "start_time": current_start,
        "end_time": current_end,
        "data_count": data_count,
        "peak_count": peak_count,
        "rr_count": rr_count,
        "sqi": sqi,
        "noise_ratio": noise_ratio,
        # Time Domain
        "ANN": ANN_val,
        "SDNN": SDNN_val,
        "SDSD": SDSD_val,
        "NN50": NN50_val,
        "pNN50": pNN50_val,
        "NN20": NN20_val,
        "pNN20": pNN20_val,
        "rMSSD": rMSSD_val,
        "MedianNN": MedianNN_val,
        # Freq Domain
        "VLF_Power": VLF_val,
        "LF_Power": LF_val,
        "HF_Power": HF_val,
        "LF_HF": LF_HF_val,
        # detail_dict (노이즈 기법별 세부 지표)
        "sigma_noise_ratio": detail_dict.get("sigma_noise_ratio"),
        "hist_noise_ratio": detail_dict.get("hist_noise_ratio"),
        "ellipse_noise_ratio": detail_dict.get("ellipse_noise_ratio"),
        "varrange_noise_ratio": detail_dict.get("varrange_noise_ratio"),
        "amplitude_noise_ratio": detail_dict.get("amplitude_noise_ratio"),
        "peakcount_noise_ratio": detail_dict.get("peakcount_noise_ratio"),
        # 그래프 경로
        "raw_plot_path": raw_plot_path,
        "filtered_plot_path": filtered_plot_path,
        "freq_plot_path": freq_plot_path
    }
    results.append(row_dict)

    current_start = current_end

# ===================================================================
# ----------------- 8. CSV 파일로 결과 저장 -------------------------
# ===================================================================
results_df = pd.DataFrame(results)

# ▼▼ CSV 저장 경로를 사용자 요청대로 변경 ▼▼
csv_path = f"/content/drive/MyDrive/RA/[wearable] 과제/HRV 분석/Datas/preprocessing/{user_name}[{min_time} ~ {max_time}].csv"
results_df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 모든 1분 구간 분석 완료! 결과 CSV: {csv_path}")
print(f"✅ 그래프 이미지는 {img_dir} 폴더에 저장되었습니다.")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



❌ 데이터 로딩 오류: [Errno 2] No such file or directory: '/content/drive/MyDrive/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승'
Traceback (most recent call last):
  File "<ipython-input-17-cce521a650a6>", line 34, in <cell line: 0>
    sensor_files = [f for f in os.listdir(data_path) if f.startswith(sensor)]
                               ^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/RA/[wearable] 과제/HRV 분석/Datas/모바일 데이터/강현승'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-cce521a650a6>", line 48, in <cell line: 0>
    sys.exit(1)
SystemExit: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local

TypeError: object of type 'NoneType' has no len()